# Latihan-3b Elastic Logs Review (Vp-Rho)

Latihan ini fokus terhadap review dan modifikasi log elastik. Asumsi koreksi invasi fluid dan borehole telah dilakukan atau akan dilakukan. Pembahasan koreksi borehole dan invasi dilakukan pada latihan berbeda
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-3b Elastic Logs Review (Vp-Rho).ipynb, accessed MM DD, YYYY."_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

__Tahap-1__ 
<br> Tetap menggunakan pre-built libraries dasar seperti numpy, matplotlib dan pandas. Sehingga kita tidak perlu membuat semua fungsi yang kita perlukan dari awal/scratch
<br> 
        
        special notes_: brugeslibrary adalah kumpulan pre-defined functions oleh Agile Geoscience bruges:
        https://github.com/agile-geoscience/bruges
        
        The bruges library's credits go to contributors: Evan Bianco, Ben Bougher, Matt Hall, Alessandro Amato del Monte, 
        Wes Hamlyn, Sean Ross-Ross

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.optimize import curve_fit
from scipy.stats import linregress
from f03elastic import elastic_sect, linerbackus
from brugeslibrary import backus, moving_average, moving_avg_fft
import matplotlib.colors as colors

__Tahap-2__
<br>Sama dengan Latihan-2, menggunakan file lokal atau yang tersimpan online untuk analisis. File path untuk lokal perlu diperhatikan dan diperbaiki jika perlu, menggunakan full path e.g. "C:/Users/Guest/Documents/python/well02logs.csv". Kemudian fungsi selanjutnya melakukan modifikasi header, pengaturan kolom, dan operasi lain, jika memang diperlukan agar mudah dibaca sebagai input analisis berikutnya. Menambahkan marker dari tabel untuk pembuatan zonasi. Karena jumlahnya yang sedikit, maka penambahan marker dapat juga menggunakan array secara manual
<br><br>_catatan_: library pandas (pd) dapat menggunakan tipe file lain seperti excel dengan command pd.read_excel('well02logs.xlsx'). Kemudian atribut fungsi reading file tersebut dapat dipelajari pada website pandas

In [ ]:
# adi2=pd.read_csv('/content/adi02logs.csv') #jika menggunakan google colab' local
adi02=pd.read_csv('adi02logs.csv', warn_bad_lines=True, index_col=0)
adi02.rename(columns={'Depth_m':'Depth', 'Vp_km/s':'Vp', 'Vs_km/s':'Vs','GammaRay_api':'GR',
                      'Density_g/cc':'Rho','Caliper_m':'Cal','Swt_v/v':'Swt','Phit_v/v':'Phit', 
                      'Vsh_v/v':'Vsh'}, inplace=True)

In [ ]:
# adi02tops=pd.read_csv('/content/adi02markers.csv') #jika menggunakan google colab' local
adi02tops=pd.read_csv('adi02markers.csv')
adi02tops.rename(columns={'Tops':'marker','Depth_m':'markerdep'}, inplace=True)
marker=np.array(adi02tops['marker'])
markerdep=np.array(adi02tops['markerdep'])
#print(adi02tops, " ", sep='\n')
#printmarker=marker.transpose()
#printmarkerdep=markerdep.transpose()

Menampilkan elastic logs yang dapat digunakan agar fokus terhadap interval of interest saja

In [ ]:
datawell=adi02
top_plot=1650
base_plot=1900
marker=marker
markerdepth=markerdep
#elastic_sect(datawell,top_plot,base_plot,marker,markerdepth)

__Tahap-3__

#### Diskusi khusus : Backus Averaging untuk menghilangkan atau mengurangi spikes dan outliers

Menggunakan fungsi backus averaging dari bruges library. Catatan dari bruges mengenai fungsi backus, sbb:
    
    Backus averaging. Using Liner's algorithm (2014):
    input:
        vp (ndarray): P-wave interval velocity.
        vs (ndarray): S-wave interval velocity.
        rho (ndarray): Bulk density.
        lb (float): The Backus averaging length in m.
        dz (float): The depth sample interval in m.
    output:
        The smoothed logs vp, vs, rho. Useful for computing other elastic parameters at a seismic scale.
    Reference:
        Liner, C (2014), Long-wave elastic attenuation produced by horizontal layering. 
        The Leading Edge, June 2014, p 634-638.

In [ ]:
logs=adi02
z=logs.index
vp0=logs.Vp # log original
vs0=logs.Vs # log original
rho0=logs.Rho # log original
length=50
sample=10
vpba,vsba,rhoba=backus(vp0, vs0, rho0, length, sample) # menjalankan fungsi backus bruges dg penjelasan di atas
vpfilt=np.zeros(np.shape(vp0))
vsfilt=np.zeros(np.shape(vs0))
rhofilt=np.zeros(np.shape(rho0))
logs['Vpfilt']=np.round(vpba,2); logs['Vsfilt']=np.round(vsba,2); logs['Rhofilt']=np.round(rhoba,2)
#with pd.option_context('expand_frame_repr', False):
#    print(logs.head(2))
#exec(open('f03plotbackus.py').read())

contoh penerapan Moving Average dan Moving Average with FFT dari bruges library yang sama

In [ ]:
length=10
vpmv=moving_average(vp0,length,mode="same")
vpmvfft=moving_avg_fft(vp0, length)
#exec(open('f03plotmvavg.py').read())

__Tahap-4__
<br>
Membuat zonasi litofasies, berdasarkan marker, cutoff vsh, porositas, saturasi. Misalnya:<br>


    sh shales 1
    sd sand   2

In [ ]:
pd.options.mode.chained_assignment = None
top_plot=1650
base_plot=1800
worklogs=logs[(logs.index >= top_plot) & (logs.index <= base_plot)]
sand=0.3; shale=0.7
sh=((worklogs.Vsh >= shale)); sd=((worklogs.Vsh <= sand))
ssflag=np.zeros(np.shape(worklogs.Vsh))
ssflag[sd.values]=1
ssflag[sh.values]=2
worklogs['ssfac']=ssflag
worklogs['AI']=round((worklogs.Vp*worklogs.Rho),2)
worklogs['SI']=round((worklogs.Vs*worklogs.Rho),2)
worklogs['VpVs']=round((worklogs.Vp/worklogs.Vs),2)
AIfilt=np.zeros(np.shape(worklogs.Vpfilt))
SIfilt=np.zeros(np.shape(worklogs.Vpfilt))
VpVsfilt=np.zeros(np.shape(worklogs.Vpfilt))
worklogs['AIfilt']=round((worklogs.Vpfilt*worklogs.Rhofilt),2)
worklogs['SIfilt']=round((worklogs.Vsfilt*worklogs.Rhofilt),2)
worklogs['VpVsfilt']=round((worklogs.Vpfilt/worklogs.Vsfilt),2)
#with pd.option_context('expand_frame_repr', False):
#    print(worklogs.head(2));

In [ ]:
warna = ['yellow','olive']
skala_fasies=colors.ListedColormap(warna[0:len(warna)], 'indexed')
f, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
ax[0].scatter(worklogs.Vp,worklogs.Rho,80,worklogs.ssfac,marker='o',
              edgecolors='k',alpha=1,cmap=skala_fasies,vmin=1,vmax=2)
ax[1].scatter(worklogs.Vpfilt,worklogs.Rhofilt,80,worklogs.ssfac,marker='o',
              edgecolors='k',alpha=1,cmap=skala_fasies,vmin=1,vmax=2)
ax[0].set_title('Vp-Rho raw', size=20)
ax[1].set_title('Vp-Rho backus filtered', size=20)
vp1=0.9*worklogs.Vp.min(); vp2=1.1*worklogs.Vp.max()
rho1=0.95*worklogs.Rho.min(); rho2=1.05*worklogs.Rho.max()
for aa in ax: 
    aa.grid(color='grey', linestyle=':')
    aa.set_xlim(vp1,vp2); aa.set_ylim(rho1,rho2)
    aa.set_xlabel('Vp', size=16); aa.set_ylabel('Rho', size=16);

## Gardner Vp-Rho (Acoustic Logs)

__Tahap-4__
<br> Hubungan empiris Power Law Gardner(1974) Vp-Rho menjelaskan hubungan rata-rata untuk banyak tipe litologi:
<br>Power Law: $ \rho = a \cdot V_p^b $
<br>Polynomial: $ \rho = a \cdot V_p^2 + b \cdot V_p + c $
<br>Ref: _(The Rock Physics Handbook, 2nd ed., p 380-381)_

Power Law (Vp.km/s dan Rho.g/cc) |
----------|

|Litho     |a    |b     |
|:-        |---: |:-:   |
|rata-rata |1.741|0.250 |
|shale     |1.75 |0.265 |
|sandstone |1.66 |0.261 |
|limestone |1.36 |0.386 |
|dolomite  |1.74 |0.252 |


Polynomial (Vp.km/s dan Rho.g/cc)|
-----------|

|Litho     |a    |b     |c   |
|:-        |---- |----: |:--:|
|shale     |-0.0261 |0.373 |1.458|
|sandstone |-0.0115 |0.261 |1.515|
|limestone |-0.0296 |0.461 |0.963|
|dolomite  |-0.0235 |0.390 |1.242|





In [ ]:
vpref=worklogs.Vpfilt
rhoref=worklogs.Rhofilt
litho=worklogs.ssfac
shale=[1.75,0.265]
sand=[1.66,0.261]
vpGshale=np.linspace(1, 6, 50)
rhoGshale=shale[0]*vpGshale**shale[1]
rhoGsand=sand[0]*vpGshale**sand[1]
# QC plot original fit
f, ax = plt.subplots(nrows=1, figsize=(8,6))
ax.scatter(vpref,rhoref,80,litho,marker='o', edgecolors='none',alpha=0.5,cmap=skala_fasies,vmin=1,vmax=2)
ax.plot(vpGshale,rhoGshale,'darkgreen', lw=4, label="Gardner shale"); 
ax.plot(vpGshale,rhoGsand,'orange', lw=4, label="Gardner sand"); 
ax.legend(loc="lower right", frameon=False)
ax.set_title('Gardner Power Law', size=20)
ax.text(3.5,1.9,'Gardner Shale a=1.75, b=0.265',color='k', size=12);
ax.text(3.5,1.8,'Gardner Sand a=1.66, b=0.261',color='k', size=12);
vp1=0.95*vpref.min(); vp2=1.05*vpref.max()
rho1=0.95*rhoref.min(); rho2=1.05*rhoref.max()
ax.grid(color='grey', linestyle=':');
ax.set_xlim(vp1,vp2); ax.set_ylim(rho1,rho2)
ax.set_xlabel('Vp', size=16); ax.set_ylabel('Rho', size=16);

In [ ]:
newshale=[1.92,0.215]
newsand=[1.63,0.296]
vpGshale=np.linspace(1, 6, 50)
newrhoGshale=newshale[0]*vpGshale**newshale[1]
newrhoGsand=newsand[0]*vpGshale**newsand[1]
# QC plot new fits
f, ax = plt.subplots(nrows=1, figsize=(8, 6))
ax.scatter(vpref,rhoref,80,litho,marker='o', edgecolors='none',alpha=0.5,cmap=skala_fasies,vmin=1,vmax=2)
ax.plot(vpGshale,newrhoGshale,'darkgreen', linestyle='dashed', lw=4, label="Fitting shale")
ax.plot(vpGshale,newrhoGsand,'orange', linestyle='dashed', lw=4, label="Fitting sand")
ax.legend(loc="lower right", frameon=False)
ax.set_title('Gardner Power Law', size=20)
labelsh=f"Shale fit a={newshale[0]:.2f}, b={newshale[1]:.2f}"
labelsd=f"Sand fit a={newsand[0]:.2f}, b={newsand[1]:.2f}"
ax.text(3.5,1.9,labelsh,color='k', size=12);
ax.text(3.5,1.8,labelsd,color='k', size=12);
vp1=0.95*vpref.min(); vp2=1.05*vpref.max()
rho1=0.95*rhoref.min(); rho2=1.05*rhoref.max()
ax.grid(color='grey', linestyle=':')
ax.set_xlim(vp1,vp2); ax.set_ylim(rho1,rho2)
ax.set_xlabel('Vp', size=16); ax.set_ylabel('Rho', size=16);

Visual fitting with varying a and b

In [ ]:
def gard(vp, a, b):
    rho = a*vp**b
    return rho
ashale=np.linspace(1.6,1.9,6)
asand=np.linspace(0.12,0.32,6)
vp=np.linspace(1, 6, 50)
f, ax = plt.subplots(nrows=1, ncols=2, figsize=(14,6))
for i in ashale:
    rhoi = gard(vp,i,0.265)
    ax1=ax[0]
    ax1.plot(vp, rhoi, '-k', linestyle='dashed', 
            lw=2, label='{:.2f}'.format(i), alpha=i-1)
    ax1.scatter(vpref,rhoref,80,litho, marker='o', 
               edgecolors='none',alpha=0.5,cmap=skala_fasies,vmin=1,vmax=2)
    ax1.set_xlim(vp1,vp2); ax1.set_ylim(rho1,rho2)
    ax1.legend(loc="lower right", frameon=False)
    ax1.set_xlabel('Vp')
    ax1.set_ylabel('Rho')
ax1.set_title('Fitting Gardner a');
for j in asand:
    rhoj = gard(vp,1.9,j)
    ax2=ax[1]
    ax2.plot(vp, rhoj, '-k', linestyle='dashed', 
            lw=2, label='{:.2f}'.format(j), alpha=j+0.4)
    ax2.scatter(vpref,rhoref,80,litho, marker='o', 
               edgecolors='none', alpha=0.5,cmap=skala_fasies,vmin=1,vmax=2)
    ax2.set_xlim(vp1,vp2); ax2.set_ylim(rho1,rho2)
    ax2.legend(loc="lower right", frameon=False)
    ax2.set_xlabel('Vp')
    ax2.set_ylabel('Rho')
ax2.set_title('Fitting Gardner b');

Curve fitting Power Law Gardner dan Polynomial Gardner (Numpy, non ML)

In [ ]:
df=worklogs
dfsh=df.loc[df['ssfac'] == 2]; dfsd=df.loc[df['ssfac'] == 1]
dfsh.head(5); #remove semicolon

In [ ]:
Vpsh1=dfsh['Vpfilt']; Rhosh1=dfsh['Rhofilt']
Vpsd1=dfsd['Vpfilt']; Rhosd1=dfsd['Rhofilt']
def powgardner(vp, a, b):
    return (a*(vp**b))
#shale power law fitting
ash1, pcov = curve_fit(powgardner,Vpsh1,Rhosh1)
a1_sh=ash1[0].round(2); b1_sh=ash1[1].round(2)
rhoshgard1=a1_sh*(vpGshale**b1_sh)
#sand power law fitting
asd1, pcov = curve_fit(powgardner,Vpsd1,Rhosd1)
a1_sd=asd1[0].round(2); b1_sd=asd1[1].round(2)
rhosdgard1=a1_sd*(vpGshale**b1_sd)
fig, axs = plt.subplots(ncols=2, figsize=(8, 4), sharey=True)
for bb in axs:
    bb.set_xticks([])
    bb.grid(color='grey', linestyle=':')
ax11 = axs[0].twiny(); ax11.plot(Vpsh1, Rhosh1, 'go')
ax11.set_title('Shale Fit', size=12)
ax12 = axs[0].twiny(); ax12.plot(vpGshale, rhoshgard1,'darkgreen', lw=2)
ax21 = axs[1].twiny(); ax21.plot(Vpsd1, Rhosd1, 'yo')
ax21.set_title('Sand Fit', size=12)
ax22 = axs[1].twiny(); ax22.plot(vpGshale, rhosdgard1,'orange', lw=2)
for cc in [ax12, ax22]:
    cc.xaxis.set_ticklabels([])
autosh=f"Shale curve fit a={a1_sh:.2f}, b={b1_sh:.2f}"
autosd=f"Sand curve fit a={a1_sd:.2f}, b={b1_sd:.2f}"
plt.text(8,3,autosh,color='k', size=14);
plt.text(8,2.8,autosd,color='k', size=14);
plt.show()
#print(f'  |  a_sh:{a1_sh:.2f}',f'b_sh:{b1_sh:.2f}', f'a_sd:{a1_sd:.2f}',f'b_sd:{b1_sd:.2f}  |  ', sep="  |  ")

In [ ]:
from numpy.polynomial import polynomial as P
Vpsh1=dfsh['Vpfilt']; Rhosh1=dfsh['Rhofilt']
Vpsd1=dfsd['Vpfilt']; Rhosd1=dfsd['Rhofilt']
#poly fitting numpy format: p(x)=c0 + c1.X + c2.X**2...cn.X**n
testcoeff, stats = P.polyfit(Vpsh1,Rhosh1,2,full=True)
print(f'diskusi: menggunakan numpy poly fit masih tidak tepat... :{testcoeff}')

<h1><center>-Akhir Program Latihan-3b-<center></h1>